<a href="https://colab.research.google.com/github/AnastasiyaPunko/TDA/blob/main/TDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Chimeric antigen receptor T cells  are a highly effective novel immune therapy.**

**Cytokine release syndrome (CRS) is the most significant and life-threatening 
toxicity after such therapy. Peak levels of some cytokines in the first month 
after infusion were highly associated with severe CRS.  We can predict which patients would develop severe CRS with a measured cytokines.**<br>

<br>
<br>
Measured cytokines (IL12,	IL13,	sIL2Ra,	MCP1,	EGF) for normal donor cohort and for patients with ALL.<br>
<br>
<br>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt

# installing giotto-tda using pip
!python -m pip install -U giotto-tda

In [ ]:
file_location = '/content/Cytokine value df.xlsx'
df = pd.read_excel(file_location)

In [ ]:
df

# **EDA**

In [ ]:
df = pd.get_dummies(df, columns=['Cohort'])

In [ ]:
type(df)

# Print dtypes and make sure that all cytokins values columns are float64
df.dtypes

In [ ]:
# Check that we don't have missing values
# hint: isna with sum
isna = df.isna().sum()
totalna = sum(isna)
print("Total NA: ", totalna)
# Assert that its actually 0 and provide some message if it's not
assert totalna == 0, print('We have missing values!')


#The values ​​of the normalized expression in this dataset cannot be less than 0. Let's check if it's true:
# Select numerical columns
# hint: select_dtypes
numcols = df.select_dtypes(include=np.number).columns
# Assert that the total number of values < 0 is 0
assert (df[numcols] < 0).any().sum() == 0, \
  "All cytokins values must be > 0"

In [ ]:
df.describe()

In [ ]:
#Histogram for each column using the built-in pandas method
_ = df.hist(figsize=(22, 18), bins=25)

In [ ]:
correlations = df.corr(method='pearson') 

plt.figure(figsize=(8.5,8.5))
sns.heatmap(correlations, square=True, annot=True, linewidths=0.25)
plt.title("Correlation matrix")
plt.show()

# **PERSISTENT HOMOLOGY**

##Generate data

In [ ]:
# Convert Pandas DataFrame Into NumPy Array
ndarray = df.to_numpy()

ndarray.shape

In [ ]:
#Gives a new shape to an array without changing its data
data=ndarray.reshape(1, *ndarray.shape)
data

In [ ]:
from gtda.plotting import plot_point_cloud
i = 0
plot_point_cloud(data[i])

##Calculate persistent homology

In [ ]:
#Instantiate a VietorisRipsPersistence transformer 
#and calculate persistence diagrams for collection of point clouds
from gtda.homology import VietorisRipsPersistence
VR = VietorisRipsPersistence()
diagrams = VR.fit_transform(data)
diagrams.shape

In [ ]:
# Persistence diagram in 2D 
from gtda.plotting import plot_diagram

i = 0
plot_diagram(diagrams[i])

##Extract features

In [ ]:
#Instantiate a PersistenceEntropy transformer and extract scalar features from the persistence diagrams.
from gtda.diagrams import PersistenceEntropy

PE = PersistenceEntropy()
features = PE.fit_transform(diagrams)

In [ ]:
features

##Use the new features in a standard classifier

In [ ]:
#from sklearn.ensemble import RandomForestClassifier
#from sklearn.model_selection import train_test_split

#X_train, X_valid, y_train, y_valid = train_test_split(features, labels)
#model = RandomForestClassifier()
#model.fit(X_train, y_train)
#model.score(X_valid, y_valid)

In [ ]:
#labels = np.zeros(40)
#labels[10:20] = 1
#labels[20:30] = 2
#labels[30:] = 3

In [ ]:
#labels

In [ ]:
#from sklearn.ensemble import RandomForestClassifier
#rf = RandomForestClassifier(oob_score=True, random_state=42)
#rf.fit(features, labels)
#rf.oob_score_

#MAPPER

##Import libraries

In [ ]:
# Data wrangling
import numpy as np
import pandas as pd  # Not a requirement of giotto-tda, but is compatible with the gtda.mapper module

# Data viz
from gtda.plotting import plot_point_cloud

# TDA magic
from gtda.mapper import (
    CubicalCover,
    make_mapper_pipeline,
    Projection,
    plot_static_mapper_graph,
    plot_interactive_mapper_graph,
    MapperInteractivePlotter
)

# ML tools
from sklearn import datasets
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA

##Generate and visualise data

In [ ]:
df=df.iloc[:,:5] 

In [ ]:
data2 = df.to_numpy()

In [ ]:
plot_point_cloud(data2)

##Configure the Mapper pipeline

In [ ]:
# Define filter function – can be any scikit-learn transformer
filter_func = Projection(columns=[0,1,2,3,4])
# Define cover
cover = CubicalCover(n_intervals=10, overlap_frac=0.3)
# Choose clustering algorithm – default is DBSCAN
clusterer = DBSCAN()

# Configure parallelism of clustering step
n_jobs = 1

# Initialise pipeline
pipe = make_mapper_pipeline(
    filter_func=filter_func,
    cover=cover,
    clusterer=clusterer,
    verbose=False,
    n_jobs=n_jobs,
)

##Visualise the Mapper graph

In [ ]:
fig = plot_static_mapper_graph(pipe, data2)
fig.show(config={'scrollZoom': True})

###Configure the colouring of the Mapper graph

In [ ]:
plotly_params = {"node_trace": {"marker_colorscale": "Blues"}}
fig = plot_static_mapper_graph(
    pipe, data2, color_data=data2, plotly_params=plotly_params
)
fig.show(config={'scrollZoom': True})

In [ ]:
# Initialise estimator to color graph by
pca = PCA(n_components=1)

fig = plot_static_mapper_graph(
    pipe, data2, color_data=data2, color_features=pca
)
fig.show(config={'scrollZoom': True})

In [ ]:
fig = plot_static_mapper_graph(
    pipe, data2, color_data=data2, color_features=pca, node_color_statistic=lambda x: np.mean(x) / 2
)
fig.show(config={'scrollZoom': True})

In [ ]:
graph = pipe.fit_transform(data2)
node_elements = graph.vs["node_elements"]
print(f"There are {len(node_elements)} nodes.\nThe first node consists of row indices {node_elements[0]}.")

In [ ]:
fig = plot_static_mapper_graph(
    pipe, data2, node_color_statistic=np.arange(len(node_elements))
)
fig.show(config={'scrollZoom': True})

###Pass a pandas DataFrame as input

In [ ]:
df2 = pd.DataFrame(data2, columns=["IL12","IL13","sIL2Ra","MCP1","EGF"])
df2.head()

In [ ]:
pipe.set_params(filter_func=Projection(columns=["IL12","IL13","sIL2Ra","MCP1","EGF"]));

In [ ]:
fig = plot_static_mapper_graph(pipe, df2, color_data=df2)
fig.show(config={'scrollZoom': True})

###Change the layout algorithm

In [ ]:
# Reset back to numpy projection
pipe.set_params(filter_func=Projection(columns=[0,1,2,3,4]));

In [ ]:
fig = plot_static_mapper_graph(
    pipe, data2, layout="fruchterman_reingold", color_data=data2
)
fig.show(config={'scrollZoom': True})

###Change the layout dimension

In [ ]:
fig = plot_static_mapper_graph(pipe, data2, layout_dim=3, color_data=data2)
fig.show(config={'scrollZoom': True})

###Change the node size scale

In [ ]:
node_scale = 30
fig = plot_static_mapper_graph(pipe, data2, layout_dim=3, node_scale=node_scale)
fig.show(config={'scrollZoom': True})